In [0]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, Conv1D, GRU
from keras.utils import to_categorical
from keras.layers import TimeDistributed
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
from matplotlib import pyplot
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
from scipy.signal import butter, filtfilt

FV = loadmat('/content/drive/My Drive/pcg.mat')
X_train = FV['FeatureVector']
print(X_train.shape)
Y_train = FV['group']
Y_train = Y_train.transpose()
print(Y_train.shape)
FVT = loadmat('/content/drive/My Drive/pcgT.mat')
X_test = FVT['FV']
print(X_test.shape)
Y_test = FVT['GP']
Y_test = Y_test.transpose()
print(Y_test.shape)
pass_band = [25*2/2000, 800*2/2000]
b, a = butter(1, pass_band, 'bandpass')
for i, sig in enumerate(X_train):
    X_train[i, :] = filtfilt(b, a, X_train[i, :])
for j, sig2 in enumerate(X_test):
    X_test[j, :] = filtfilt(b, a, X_test[j, :])
X = np.concatenate((X_train, X_test))
Y = np.concatenate((Y_train, Y_test))
print(X.shape)
print(Y.shape)

X = X.reshape(3541, 10000, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)
# fit and evaluate a model
Y_train = to_categorical(Y_train, num_classes=2)
Y_test = to_categorical(Y_test, num_classes=2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 1, 5000, 512
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 100, 100
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    print(trainX.shape)
    print(testX.shape)
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=8, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=4, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=2, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))    
    model.add(Dropout(0.5))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))  
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(512))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # print(model.summary())
    # fit network
    #model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    model.fit(x=trainX, y=trainy, batch_size=batch_size, epochs=epochs, verbose=verbose, callbacks=None, validation_data=(testX, testy),
              use_multiprocessing=True)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10):
    # load data
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(X_train, Y_train, X_test, Y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

# run the experiment
run_experiment()